#Importing Libratires

In [1]:
import pandas as pd
import random
import re
import ast
import spacy

In [2]:
%cd '/content/drive/My Drive/Written Test in SKS'

/content/drive/My Drive/Written Test in SKS


In [3]:
!ls

A-InternCodingTest-DataScience.pdf  coupons_ner.csv  work_sample.txt
Annotated_work_sample.txt	    PreTask.ipynb


#Importing Dataset

In [4]:
data = pd.read_csv('coupons_ner.csv', header=None)
data.columns = ['OfferDetails']
data

,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...
...,...
881,SAVE $2.00 when you buy $7 worth of participat...
882,SAVE $5.00 on any Gain scented purchase of $30...
883,SAVE $1.00 when you buy any ONE (1) NEW AXE pr...
884,BUY TWO GET ONE Buy ONE (1) CHIPS AHOY! Cookie...


## First five offer details

In [5]:
print('First 5 OfferDetails: ')
for i in range(5):
    print(data['OfferDetails'][i])

First 5 OfferDetails: 
Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).
Save $2.00 ONE Tide PODS OR Tide Power PODS (excludes Tide Liquid/Powder Laundry Detergent, Tide Simply, Tide Simply PODS, and Tide PODS 9 ct and below, and trial/travel size).
Save $2.00 ONE Tide Laundry Detergent (excludes Tide Purclean Laundry Detergent, Studio by Tide Laundry Detergent, Tide PODS, Tide Rescue, Tide Simply Laundry Detergent, Tide Simply PODS, Tide Detergent 10 oz and trial/travel size).
SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 - 15.2 OZ. EXCLUDES LARGE SIZE) any flavor/variety G

## Extracting Work Sample

Here we are taking sample of 100 random points which will help us to evaluate our model. Annotating 888 points is really time consuming thus selected a sample and annotated it manually

In [6]:
work_sample = data.sample(100)

In [ ]:
with open('work_sample.txt', 'w') as ws:
    for detail in work_sample:
        ws.write(detail+'\n')

## Annotated Work Sample

In [15]:
df = pd.read_csv('Annotated_work_sample.txt', sep='|')
df

,Product,FaceValue,OfferDetails
0,liquid laundry detergent,$2.00,Save $2.00 off ONE all liquid laundry detergen...
1,Gardein Breakfast Bowl,$1.00,Save $1.00 on Gardein Breakfast Bowl When you ...
2,Family Pet Meaty Cuts Dry Dog Food,$1.00,$1.00 OFF Family Pet Meaty Cuts Dry Dog Food 2...
3,"Gain Liquid Fabric Softener,Gain Fireworks",NaN,Gain ONE Gain Liquid Fabric Softener 48 ld OR ...
4,"Gain Flings,Gain Liquid Laundry Detergent,Gain...",$1.00,Save $1.00 on ONE Gain Flings 12 ct TO 26 ct O...
...,...,...,...
95,MAXWELL HOUSE Iced Coffee can,$0.50,SAVE $0.50 off ONE (1) MAXWELL HOUSE Iced Coff...
96,[],NaN,Chobani Greek Yogurt 32-oz.
97,[],NaN,Signature SELECT Cheese Crunchies 8-oz. Limit 1.
98,Dark Chocolate Dessert Dip,$0.75,SAVE $0.75 on ONE (1) Sabra® Dark Chocolate De...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product       100 non-null    object
 1   FaceValue     59 non-null     object
 2   OfferDetails  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


#Exercise 1


Write a function that takes OfferDetails as input, and returns FaceValue as output

In [ ]:
def FaceValue(detail):
    fv = re.findall(r'(\$\d*(?:\.\d+)?)', detail)
    if len(fv)==0:
        return float('NaN')
    else:
        return fv[0]

OfferDet = random.sample(list(df['OfferDetails']),1)[0]
FaceVal = FaceValue(OfferDet)

print('OfferDetail is: ', OfferDet)
print('Extracted FaceValue is: ', FaceVal)

OfferDetail is:  "Save $2.00 On Gatorade Sports Drink or Propel Water When you Buy TEN (10), Any Variety, Gatorade 1-qt. btl., Propel  Water Beverage, 20-24 oz. btl. (Plus Deposit or Fee Where Required)-Mix or Match!"
Extracted FaceValue is:  $2.00


Run the function against all rows in the attached data and determine function accuracy (Higher the better)

In [ ]:
ExtractedFaceValues = df['OfferDetails'].apply(FaceValue)
GTFaceValues = df['FaceValue']
match_cnt = 0
for i in range(len(GTFaceValues)):
    if(ExtractedFaceValues[i]==GTFaceValues[i] or (pd.isna(ExtractedFaceValues[i]) and pd.isna(GTFaceValues[i]))):
        match_cnt+=1
       
print('Accuracy: {:.2f}%'.format(100*(match_cnt/len(GTFaceValues))))

Accuracy: 100.00%


#Exercise 2

Write a function that takes OfferDetails as input, and returns Product as output (Product can be single or array)

Here we are loading pre trained Name Entity Recognition model using spacy. As we dont have annotated data thus fine tuning is really defficult that is why taken this approach. In our offerDetail there is only product name and some product company name. When we predict name entity then lots of entity get labbeled as PERSON, WORK_OF_ART etc which is actually a Product name thus we are considering all of the as a Product name to increase the accuracy

In [8]:
def preprocess(text):
    #Removing digits and some special characters like ./()
    text = re.sub('[0-9\$\(\)\.\,]+', '', text)
    return text

In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
def Product(detail):
    doc = nlp(detail)
    products = []
    for ent in doc.ents[:]:
        if(ent.label_ in ['PRODUCT','ORG','LOC','WORK_OF_ART','FAC','EVENT','PERSON']):
            products.append(ent.text)
    return products

In [11]:
OfferDet = data['OfferDetails'][0]
pre_OfferDet = preprocess(OfferDet)
product = Product(pre_OfferDet)

print('OfferDetail is: ', OfferDet)
print('Extracted Product(s) are: ', product)

OfferDetail is:  Save $2.00 ONE Downy Liquid Fabric Conditioner 72 ld or larger (includes Downy Odor Protect 48 oz or larger OR Downy Wrinkle Guard 40 oz or larger OR Downy Nature Blends 67 oz or larger) OR Bounce/Downy Sheets 130 ct or larger (includes Bounce/Downy Wrinkle Guard 80 ct or larger) OR In Wash Scent Boosters 8.6 oz or larger (includes Downy Unstopables, Fresh Protect, Odor Protect, and Infusions) (excludes Downy Libre Enjuague, Gain Fireworks, and trial/travel size).
Extracted Product(s) are:  ['Downy Liquid Fabric Conditioner', 'Downy Odor Protect', 'Downy Wrinkle Guard', 'Downy Nature Blends', 'Bounce/Downy Sheets', 'Bounce/Downy Wrinkle Guard', 'Downy Unstopables Fresh Protect Odor Protect', 'Downy Libre', 'Gain Fireworks']


Run the function against all rows in the attached data and determine function accuracy (Higher the better)

In [29]:
ExtractedProducts = df['OfferDetails'].apply(preprocess).apply(Product)
GTProduct = df['Product'].apply(lambda x: x.split(','))
acc = 0
for i in range(len(GTProduct)):
    acc += len((set(ExtractedProducts[i]).intersection(set(GTProduct[i]))))/len(set(GTProduct[i]))
       
print('Accuracy: {:.2f}%'.format(100*(acc/len(GTProduct))))

Accuracy: 17.66%
